# CS3263 Assignment 1 - Problem 1: Friendly Martian Hero

Welcome to the CS3263 Assignment 1 - Problem 1, you will complete the problem in this notebook!

* Group Member 1:
    - Name:
    - Student ID:


* Group Member 2:
    - Name:
    - Student ID:


## Overview

In problem 1, we are going to investigate an important AI paradigm - logical representation and inference. We will develop a knowledge base in first order logic, and draw interesting inferences using Forward Chaining and Backward Chaining algorithms - the foundations of AI rule-based systems that are still very useful in practice.

- Constructing sentences using: `expr`
- Creating first-order-logic knowledge bases using: `FolKB`
- Inference in first order knowledge base
    - Unification
    - Forward chaining algorithm
    - Backward chaining algorithm

## Your Tasks

You only need to submit the answers to Task 1, Task 2, and Task 3 in this problem 1. (But you will need to work through and understand the whole program to complete the tasks.)

You are expected to add your codes **ONLY** in the blocks noted by:

```python
'''---Your code starts here---'''

'''---Your code ends here----'''
```

Let's start!


## Programming Environment Setup

First, we will set up the programming environment by importing one package included in the Assignment Files.

In [ ]:
from FAIAss1.aimalogic4e import *

Let's go through the basics of the logic module.

## `expr`: A Shortcut for Constructing Sentences

You can use the function `expr` to specify statements in a knowledge base of first-order logic sentences.

In [ ]:
# Example
expr('~(P & Q)  ==>  (~P | ~Q)')

In [ ]:
# Example
expr("(Dog(x) & Owns(Alice, x)) ==> Loves(Alice, x)")

In [ ]:
# Example
expr('sqrt(b ** 2 - 4 * a * c)')

## `FolKB`: First-Order Logic Knowledge Bases

The class `FolKB` can be used to represent a knowledge base of First-order logic sentences. The clauses are first-order definite clauses. We will see how to write such clauses to create a database and query them in the following sections.

### Mars Adventure KB

In this section we create a `FolKB` based on the following paragraph.

*A Mars Rover would become a hero if it receives some earthly souvenirs from nice Martians. Mr. Green, a nice Martian (to Rovers), has some earthly souvenirs (some colourful pairs of slippers), and he gave some to the Mars Rover Persy. (Note: If the Martian were not nice, the Rover won't be around to become a "hero").*

The first step is to extract the facts and convert them into first-order definite clauses. Extracting the facts from data alone is a challenging task. Fortunately, we have a short story and we can do extraction and conversion manually. We'll store the clauses in list aptly named `clauses`.

In [ ]:
clauses = []

Assume the following predicate symbols: 

* `Hero(x)`: `x` is a Hero (Rover)
* `Rover(x)`: `x` is a Rover
* `Martian(x)`: `x` is a Martian
* `Gives(x, y, z)`: `x` gives `y` to `z` **(Note: order matters in this expression -- assume `x` is a Martian, `y` is a souvenir, and `z` is a Rover here)**
* `Souvenir(x)`: `x` is a souvenir
* `Slippers(x)`: `x` is a pair of slippers
* `Nice(x)`: `x` is (a) Nice (Martian)
* `Friend(x, y)`: `x` is friendly to `y` **(Note: order matters in this expression -- assume `x` is a Martian and `y` is a Rover here)**.
* `Owns(x, y)`: `x` owns `y` **(Note: order matters in this expression -- assume `x` is a Martian and `y` is a souvenir here)**.

Let's now combine them with appropriate variable naming to depict the meaning of the facts to put into the knowledge base.

We know that:  

1. Mr. Green, the Martian is a friend of Persy, the Rover.

Represent these individuals using the constant symbols `Green` and `Persy`. The friend relation is shown using the predicate symbol `Friend`.

- $\text{Martian}(\text{Green})$
- $\text{Rover}(\text{Persy})$
- $\text{Friend}(\text{Green}, \text{Persy})$

In [ ]:
clauses.append(expr("Martian(Green)"))
clauses.append(expr("Rover(Persy)"))
clauses.append(expr("Friend(Green, Persy)"))

We also know that:

2. Green has some earthly souvenirs

This states the existence of some souvenir which is owned by Green. 

$\exists x \text{Owns}(\text{Green}, x) \land \text{Souvenir}(x)$. 

We invoke existential instantiation to introduce a new constant `S1` which is the souvenir owned by Green.

$\text{Owns}(\text{Green}, \text{S1}), \text{Souvenir}(\text{S1})$

In [ ]:
clauses.append(expr("Owns(Green, S1)"))
clauses.append(expr("Souvenir(S1)"))

### Task 1: Building the KB 

Express the rest of the sentences as definite clauses and add to the `clauses` list.

**Hints:** Some clauses you may want to include:

* $\text{Souvenir}(x) \land \text{Owns}(x, y)\land \text{Friend}(x, z) \implies \text{Gives}(x, y, z)$
* $\text{Slippers}(x) \implies \text{Souvenir}(x)$
* $\text{Martian}(x) \land \text{Rover}(y) \land \text{Friend}(x, y) \implies \text{Nice}(x)$


In [ ]:
# --- Problem 1 Task 1: your code starts here---

# clauses.append(expr("..."))

# --- Problem 1 Task 1: your code ends here---

print(clauses)

## Creating the Knowledge Base

Now that we have converted the information into first-order definite clauses we can create our first-order logic knowledge base.

In [ ]:
mars_kb = FolKB(clauses)

### Operations on the Knowledge Base

You can access the clauses in the `KB` by the following methods: `tell`, `ask`, `ask_generator`, `retract`.

Evaluate the following to see the source definitions of the methods. 

In [ ]:
psource(KB)

The `print_kb` helper function shows the content of the KB

In [ ]:
def print_kb(kb):
    for item in kb.clauses:
        print(item)

In [ ]:
print_kb(mars_kb)

The `subst` helper function substitutes variables with given values in first-order logic statements.
This will be useful in later algorithms. Its implementation is quite simple and self-explanatory.

In [ ]:
psource(subst)

Here's an example of how `subst` can be used.

In [ ]:
# Example
subst({x: expr('Green'), y: expr('S1')}, expr('Owns(x, y)'))

## Inference in First-Order Logic

In this section we look at a **forward chaining** and a **backward chaining** algorithm for `FolKB`. Both aforementioned algorithms rely on a process called **unification**, a key component of all first-order inference algorithms.

### Unification

We sometimes require finding substitutions that make different logical expressions look identical. This process, called unification, is done by the `unify` algorithm. It takes as input two sentences and returns a *unifier* for them if one exists. A unifier is a dictionary which stores the substitutions required to make the two sentences identical. It does so by recursively unifying the components of a sentence, where the unification of a variable symbol `var` with a constant symbol `Const` is the mapping `{var: Const}`. Let's look at a few examples.

In [ ]:
# Example
unify(expr('x'), 3)

In [ ]:
# Example
unify(expr('A(x)'), expr('A(B)'))

In [ ]:
# Example
unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(y)'))

In cases where there is no possible substitution that unifies the two sentences the function return `None`.

In [ ]:
# Example
print(unify(expr('Cat(x)'), expr('Dog(Dobby)')))

We also need to take care we do not unintentionally use the same variable name. Unify treats them as **a single variable** which prevents it from taking multiple value.

In [ ]:
# Example
print(unify(expr('Cat(x) & Dog(Dobby)'), expr('Cat(Bella) & Dog(x)')))

### Forward Chaining Algorithm

We consider the simple forward-chaining algorithm. We look at each rule in the knowledge base and see if the premises can be satisfied. This is done by finding a substitution which unifies each of the premise with a clause in the `KB`. If we are able to unify the premises, the conclusion (with the corresponding substitution) is added to the `KB`. This inference process is repeated until either the query can be answered or till no new sentences can be added. We test if the newly added clause unifies with the query in which case the substitution yielded by `unify` is an answer to the query. If we run out of sentences to infer, this means the query was a failure.

The function `fol_fc_ask` is a generator which yields all substitutions which validate the query.

In [ ]:
psource(fol_fc_ask)

### Backward Chaining Algorithm

This algorithm works backward from the goal, chaining through rules to find known facts that support the proof. Suppose `goal` is the query we want to find the substitution for. We find rules of the form $\text{lhs} \implies \text{goal}$ in the `KB` and try to prove `lhs`. There may be multiple clauses in the `KB` which give multiple `lhs`. It is sufficient to prove only one of these. But to prove a `lhs` all the conjuncts in the `lhs` of the clause must be proved. This makes it similar to *And/Or* search.

#### Ask

The function `fol_bc_ask` is a generator which yields all substitutions which validate the query.

In [ ]:
psource(fol_bc_ask)

#### OR

The *OR* part of the algorithm comes from our choice to select any clause of the form $\text{lhs} \implies \text{goal}$. Looking at all rules `lhs` whose `rhs` unify with the `goal`, we yield a substitution which proves all the conjuncts in the `lhs`. We use `parse_definite_clause` to attain `lhs` and `rhs` from a clause of the form $\text{lhs} \implies \text{rhs}$. For atomic facts the `lhs` is an empty list.

In [ ]:
psource(fol_bc_or)

#### AND

The *AND* corresponds to proving all the conjuncts in the `lhs`. We need to find a substitution which proves each *and* every clause in the list of conjuncts.

In [ ]:
psource(fol_bc_and)

### Task 2: Who Are My Friends? 

#### Task 2-1: Add Good News

It's a good day - Persy makes another new friend, Ms. Grassly (He is a Martian, of course)! Add that news to the `mars_kb`.

**Hints:** You can use the following functions to access `mars_kb`:

* To add an expression to the `KB`:
    `<KB>.tell(<expr>)`
* To ask about an expression in KB:
    ```
    answer = <ask_function>(<KB>, <expr>)
    ```
    where `<ask_function>` is `fol_fc_ask` for forward chaining or `fol_bc_ask` for backward chaining.

In [ ]:
# Add news to KB:

# --- Problem 1 Task 2.1: your code starts here---


# --- Problem 1 Task 2.1: your code ends here---

print_kb(mars_kb)

#### Task 2-2: Find Nice Martians

Ask the knowledge base `mars_kb` about all the nice Martians that Persy has met. 

Explore **both** the forward chaining algorithm `fol_fc_ask` and backward chaining algorithm `fol_bc_ask`.

**Please note:** After calling one of the chaining algorithm, you have to **reset** the KB to its "original state" as in the problem definition (to remove the added facts, if any), before calling the other to get the "clean results" for each round.

In [ ]:
# Forward chaining algorithm:

# --- Problem 1 Task 2.2-1: your code starts here---

answer_fc = None

# --- Problem 1 Task 2.2-1: your code ends here---

print(list(answer_fc))

After the forward chaining algorithm, we should **refresh the KB** to its original state.

In [ ]:
mars_kb = FolKB(clauses)

Remember to add the news to the KB again.

In [ ]:
# Add news to KB:

# --- Problem 1 Task 2.2-2: your code starts here---


# --- Problem 1 Task 2.2-2: your code ends here---

print_kb(mars_kb)

In [ ]:
# Backward chaining algorithm:

# --- Problem 1 Task 2.2-2: your code starts here---

answer_bc = None

# --- Problem 1 Task 2.2-3: your code ends here---

print(list(answer_bc))

## Task 3: Who is the HERO? 

The definition of a **Hero** as follows:

* The Hero `z` is the Rover `z` who receives souvenir `y` from a **nice** Marian `x`. (i.e., Nice Martian `x` gives the souvenir `y` to Rover `z`, and `z` becomes the Hero.)

### Task 3-1: Add Hero Definition

Add any additional clause(s) to the `mars_kb` as necessary to define a Hero.


In [ ]:
# --- Problem 1 Task 3-1: your code starts here---



# --- Problem 1 Task 3-1: your code ends here---

print_kb(mars_kb)

### Task 3-2: Find the Hero

Use **both** forward chaining and backward chaining to find out who is the Hero? Similarly, remember to **reset the KB** to its original state before calling the other algorithm.

In [ ]:
# Find Hero with Forward Chaining and Backward Chaining

# --- Problem 1 Task 3-2: your code starts here---

answer_fc = None

answer_bc = None

# --- Problem 1 Task 3-2: your code ends here---

print(list(answer_fc))
print(list(answer_bc))

### Conclusion and Lessons Learned

In this problem, we explored two fundamental AI inference techniques: forward chaining and backward chaining, applied within a first-order logic knowledge base. Forward chaining, a data-driven approach, systematically derives new facts, while backward chaining, a goal-driven method, works backward from queries to validate conditions. By comparing both approaches, we gained insights into their efficiency, applicability, and differences in reasoning.

We also developed hands-on experience with knowledge representation, unification, and inference rule application. These exercises underscored the importance of structured logical reasoning in AI and demonstrated how rule-based systems can model complex relationships and support automated decision-making.

## Submission

After the completion of the tasks, **copy** your solutions to the `./solution1.py` file indicated by the respective placeholders in the file.

End of Problem 1.

### Acknowledgments

The programming assignment is adapted from a Jupyter notebook supporting the book *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)*. We make use of the implementations in the [logic.py](https://github.com/aimacode/aima-python/blob/master/logic.py) module,together with some utility functions.  

Content adapted from notebook on AIMA website created by [Chirag Vartak](https://github.com/chiragvartak) and [Peter Norvig](https://github.com/norvig).